In [8]:
import numpy as np
import matplotlib.pyplot as plt
import timeit
from numba import prange
from numba import njit
from matplotlib import animation

# Part 1
Below we are initialising our 2D grid. We simulate the discrete diffusion process on this grid. 

In [9]:
N = 100

un = np.zeros((N+2,N+2), dtype=np.float64)
#un[1,1]=1


In [10]:
def diffusion_iteration(un):
    un1 = np.full_like(un, un[0,0])
    for index, un_ij in np.ndenumerate(un[1:-1, 1:-1]):
        i,j =index[0],index[1]
        un1[i+1,j+1] = (un[i,j+1]+un[i+2,j+1]+un[i+1,j]+un[i+1,j+2])/4
    return un1


In [11]:
print(diffusion_iteration(un))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [12]:
q0=%timeit -o diffusion_iteration(un)

27.1 ms ± 1.47 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
qavg=q0.average
qdev=q0.stdev

In [ ]:
print(qdev)

In [ ]:
Ns = [10,20,30,40,50,60,70,80,90,100]
t_avgs = []
t_stdevs = []
for N_i in Ns:
    un_i = np.zeros((N_i+2,N_i+2), dtype=np.float64)
    t = %timeit -o diffusion_iteration(un_i)
    t_avgs.append(t.average)
    t_stdevs.append(t.stdev)
    

In [ ]:
print(t_avgs)
print(t_stdevs)

In [ ]:
plt.errorbar(Ns, t_avgs, t_stdevs)
plt.ylabel('Time /s')
plt.xlabel('N')

In [ ]:
@njit
def jit_diffusion_iteration(un):
    un1 = np.full_like(un, un[0,0])
    for index, un_ij in np.ndenumerate(un[1:-1, 1:-1]):
        i,j =index[0],index[1]
        un1[i+1,j+1] = (un[i,j+1]+un[i+2,j+1]+un[i+1,j]+un[i+1,j+2])/4
    return un1

In [ ]:
q1=%timeit -o jit_diffusion_iteration(un)

In [ ]:
Ns = [10,20,30,40,50,60,70,80,90,100]
t_avgs = []
t_stdevs = []
for N_i in Ns:
    un_i = np.zeros((N_i+2,N_i+2), dtype=np.float64)
    t = %timeit -o jit_diffusion_iteration(un_i)
    t_avgs.append(t.average)
    t_stdevs.append(t.stdev)

plt.errorbar(Ns, t_avgs, t_stdevs)
plt.ylabel('Time /s')
plt.xlabel('N')

In [ ]:
un = np.zeros((N+2,N+2), dtype=np.float64)
#un[2,3]=1
@njit
def prange_diffusion_iteration(un):
    un1 = np.full_like(un, un[0,0])
    for i in prange(N):
        for j in prange(N):
            un1[i+1,j+1] = (un[i,j+1]+un[i+2,j+1]+un[i+1,j]+un[i+1,j+2])/4
    return un1
            

In [ ]:
prange_diffusion_iteration(un)

In [ ]:
q2=%timeit -o prange_diffusion_iteration(un)

In [ ]:
fig = plt.figure()
ax = plt.axes()
image= ax.imshow(un)

In [ ]:
def init():
    image.set_data(np.empty((1,1)))
    return image,

In [ ]:

def animate(input_1,un):
    
    un1 = np.full_like(un, un[0,0])
    for i in prange(N):
        for j in prange(N):
            un1[i+1,j+1] = (un[i,j+1]+un[i+2,j+1]+un[i+1,j]+un[i+1,j+2])/4
    image.set_data(un1)
    image.autoscale()
     
    un = un1.copy()
    return image,

In [ ]:
qua = [1,2,3]

def f():
    global qua
    qua = [1,1,1]

print(qua)

In [ ]:
f()
print(qua)

In [ ]:
un = np.zeros((N+2,N+2), dtype=np.float64)
un[30:-30,30:-30]=1
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=200, interval=20, blit=True)
anim.save('assignment_1_animation.mp4', fps=30, extra_args=['-vcodec', 'libx264'])
plt.show()